In [1]:
# 需要模組
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, Sequential, metrics
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 設定資料夾
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/0609財管課程')

Mounted at /content/drive


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
# preprocess
# 轉換色彩 0~255 資料為 0~1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# y 值轉成 one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [5]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=32,即 output space 的深度, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss=tf.keras.losses.categorical_crossentropy,
       optimizer=tf.keras.optimizers.Adadelta(),
       metrics=['accuracy'])

# 進行訓練, 訓練過程會存在 train_history 變數中
'''
train_history = model.fit(x_train, y_train,
              batch_size=100,
              epochs=10,
              verbose=1,
              validation_data=(x_test, y_test))
'''
train_history = model.fit(x_train, y_train,
              batch_size=100,
              epochs=10,
              verbose=1,
              validation_split=0.2)


Epoch 1/10
480/480 [==============================] - 15s 9ms/step - loss: 2.2932 - accuracy: 0.1295 - val_loss: 2.2730 - val_accuracy: 0.3083
Epoch 2/10
480/480 [==============================] - 4s 8ms/step - loss: 2.2612 - accuracy: 0.2345 - val_loss: 2.2354 - val_accuracy: 0.4665
Epoch 3/10
480/480 [==============================] - 3s 7ms/step - loss: 2.2230 - accuracy: 0.3288 - val_loss: 2.1882 - val_accuracy: 0.5433
Epoch 4/10
480/480 [==============================] - 4s 7ms/step - loss: 2.1735 - accuracy: 0.3992 - val_loss: 2.1254 - val_accuracy: 0.5972
Epoch 5/10
480/480 [==============================] - 4s 7ms/step - loss: 2.1053 - accuracy: 0.4584 - val_loss: 2.0390 - val_accuracy: 0.6428
Epoch 6/10
480/480 [==============================] - 4s 7ms/step - loss: 2.0155 - accuracy: 0.4983 - val_loss: 1.9219 - val_accuracy: 0.6834
Epoch 7/10
480/480 [==============================] - 4s 7ms/step - loss: 1.8965 - accuracy: 0.5334 - val_loss: 1.7697 - val_accuracy: 0.7167
Epoch

In [6]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.2223 - accuracy: 0.7871


[1.2223377227783203, 0.7871000170707703]

### 一維資料 股票市場

In [37]:
import os
import pandas as pd

df = pd.read_csv('./2330TSM.csv', encoding='utf-8')
#print(df)
data = df.iloc[:-1,4:].values
label = df.iloc[:,3].values
label = [ 1 if k>label[idx-1] else 0 for idx, k in enumerate(label[1:])]
label = np.array(label)
print(data.shape)
print(label.shape)

(243, 5)
(243,)


In [44]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
scaler = MinMaxScaler() #最小最大
data = scaler.fit_transform(data)
#X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=0)
X_train = data[:-40]
y_train = label[:-40]
X_test = data[-40:]
y_test = label[-40:]

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam


input = Input(shape=(5, 1))
x = Conv1D(64, 2, activation="relu")(input)
x = Dense(16, activation="relu")(x)
x = MaxPooling1D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation = 'sigmoid')(x)
model = Model(input, output)
model.compile(optimizer='adam', loss="binary_crossentropy", metrics = ['accuracy'])
model.summary()

model.fit(X_train, y_train, batch_size=16,epochs=10, verbose=1, validation_split=0.2)

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 5, 1)]            0         
                                                                 
 conv1d_11 (Conv1D)          (None, 4, 64)             192       
                                                                 
 dense_38 (Dense)            (None, 4, 16)             1040      
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 2, 16)            0         
 g1D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 32)                0         
                                                                 
 dense_39 (Dense)            (None, 256)               8448      
                                                          

In [46]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 0.6965 - accuracy: 0.5000


[0.696506142616272, 0.5]

In [32]:
import os
import pandas as pd

df = pd.read_csv('./2330TSM.csv', encoding='utf-8')
#print(df)
data = df.iloc[:-1,4:].values
label = df.iloc[:,3].values

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
scaler = MinMaxScaler() #最小最大
data = scaler.fit_transform(data)
X_train = data[:-80]
y_train = label[:-80]
X_test = data[-80:]
y_test = label[-80:]

In [34]:
# build modeel
model = Sequential()
model.add(layers.SimpleRNN( input_shape=(5, 1) , units=256 ))
model.add(layers.Dense(1))
model.compile(optimizer='adam',loss='mse')
history = model.fit(X_train ,y_train, epochs=50, validation_split=0.2, shuffle=False)

Epoch 1/50
5/5 [==============================] - 1s 61ms/step - loss: 352870.8750 - val_loss: 391500.1250
Epoch 2/50
5/5 [==============================] - 0s 13ms/step - loss: 346472.9688 - val_loss: 381667.1875
Epoch 3/50
5/5 [==============================] - 0s 15ms/step - loss: 337430.1562 - val_loss: 376058.6562
Epoch 4/50
5/5 [==============================] - 0s 11ms/step - loss: 332508.9062 - val_loss: 372852.0938
Epoch 5/50
5/5 [==============================] - 0s 11ms/step - loss: 329788.3125 - val_loss: 370628.1875
Epoch 6/50
5/5 [==============================] - 0s 12ms/step - loss: 327828.8438 - val_loss: 368735.2188
Epoch 7/50
5/5 [==============================] - 0s 12ms/step - loss: 326063.9062 - val_loss: 366896.3125
Epoch 8/50
5/5 [==============================] - 0s 11ms/step - loss: 324340.0000 - val_loss: 365075.8125
Epoch 9/50
5/5 [==============================] - 0s 11ms/step - loss: 322627.6250 - val_loss: 363277.1250
Epoch 10/50
5/5 [====================